# Create Latex Tables with Enemy Wave Data

Create a bunch of latex tables containing the wave data.

## Extract the level data from the source files

In [336]:
"""
Pull out all the level data from the source files.
"""
files = ["../iridisalpha/src/level_data/level_data.asm",
         "../iridisalpha/src/level_data/level_data2.asm"]
d = {}
level_data = {}
ch = None
ck = None
for f in files:
    ld = open(f, 'r')
    while True:
        l = ld.readline()
        if not l:
            break
        if "pieceOfPlanet" in l:
            continue
        if l[:2].strip():
            if ch and d:
                level_data[ch] = d
            ch = l.split()[0].strip()
            d = {}
            continue

        k = l[10:17]
        if "Byte" in k:
            ck = k
            continue

        if "BYTE" in l:
            v = l[14:].strip().split(',')[0].replace('<','')
            d[ck] = v

# Drop any spurious entries
level_data = {k:c for k,c in level_data.items() if len(c) == 30}
len(level_data.keys())

200

## Look at unused data

In [337]:
# Get the settings which are never used.

def createSimpleTable(cands):
    cols = list(list(cands.values())[0].keys())
    tb = []
    for p,d in level_data.items():
        tb += [list(d.values())]
    return (tb, cols)

level_table,cols = createSimpleTable(level_data)
level_fields = [l for l in level_table]
for i in range(0,max([len(l) for l in level_fields])):
        m = set([l[i] for l in level_fields])
        if len(m) > 3:
            continue
        print(cols[i], sorted(m))

Byte 11 ['nullPtr']
Byte 12 ['$00']
Byte 14 ['nullPtr']
Byte 15 ['$00', '$01']
Byte 33 ['$00', '$60', '$88']
Byte 34 ['$00', '$14']


In [348]:
#l = ["Byte 15", "Byte 34", "Byte 33", "Byte 7 "]
l = ["Byte 7 ", "Byte 9 "]
for k,c in level_data.items():
    b7 = c["Byte 7 "]
    b9 = c["Byte 9 "]
    if b7 != "$00" and b9 != "nullPtr":
        print(k, b7,b9)

planet3Level4Data $04 gilbyLookingLeft
planet3Level6Data $04 planet3Level6Additional
planet5Level5Data $05 planet5Level5Additional
fighterShipAsExplosion $01 planet3Level10Data
planet2Level19Data $0C landGilbyAsEnemy
planet5Level14Data $06 llamaWaveData
planet1Level11Data $03 smallDotWaveData
smallDotWaveData $01 smallDot2WaveData
planet1Level14Data $03 planet1Level8Data
planet5Level3Data $01 planet5Level3Additional
planet5Level3Data2ndStage $01 planet5Level3Additional
planet4Level19Data $01 planet4Level19Additional


In [345]:
#l = ["Byte 15", "Byte 34", "Byte 33", "Byte 7 "]
l = ["Byte 7 ", "Byte 9 "]
for k,c in level_data.items():
    for i in l:
        if c[i] != "$00":
            print(i, k, c[i])

Byte 7  planet4Level19Additional $00
Byte 7  planet4Level19Additional2ndWave $00
Byte 7  planet2Level7Data $00
Byte 7  planet2Level7Data2ndStage $00
Byte 7  planet4Level4Data $00
Byte 7  planet4Level4Data2ndStage $00
Byte 7  planet4Level3Data $00
Byte 7  planet4Level2Data2ndStage $00
Byte 7  planet1Level9Data $00
Byte 7  planet1Level9DataSecondStage $00
Byte 7  secondExplosionAnimation $00
Byte 7  planet3Level3Data $00
Byte 7  planet3Level3Data2ndStage $00
Byte 7  planet3Level3Data3rdStage $00
Byte 7  planet2Level8Data $00
Byte 7  planet2Level8Data2ndStage $00
Byte 7  planet2Level8Data3rdStage $00
Byte 7  planet2Level9Data $00
Byte 7  gilbyTakingOffAsExplosion $00
Byte 7  gilbyLookingLeft $00
Byte 7  planet3Level4Data $04
Byte 9  planet3Level4Data gilbyLookingLeft
Byte 7  planet3Level5Data $00
Byte 7  stickyGlobeExplosion $00
Byte 7  planet3Level6Data $04
Byte 9  planet3Level6Data planet3Level6Additional
Byte 7  planet3Level6Additional $00
Byte 7  planet1Level17Data $00
Byte 7  planet1

## Create tables for main wave data

In [221]:
import pandas as pd
from tabulate import tabulate
from texttable import Texttable
import latextable


styles = [dict(selector="caption", 
    props=[("text-align", "center"),
    ("font-size", "120%"),
    ("color", 'black')])]

def createTable(indices, cands):
    """ Make a table and column list of the wave data"""

    tb = []
    cols = ['Planet', 'Level'] 
    all_cols = list(list(cands.values())[0].keys())
    for i in indices:
        cols += [all_cols[i]]
    for p,d in cands.items():
        t = p.replace('planet','').replace('Level', ' ').replace('Data', '')
        f = [*t.split()]
        l = list(d.values())
        for i in indices:
            if i >= len(l):
                continue
            f += [l[i]]
        tb += [f]
    level_table = sorted(tb, key=lambda x: (x[0],int(x[1])))
    return (level_table, cols)


In [248]:
# Restrict `cands` to just the guys that are the initial wave data for each
# wave. We'll look at the others separately.
cands = {c:v for c,v in sorted(level_data.items())
         if c.startswith("planet") and c.endswith("Data")}

all_fields = list(range(0,30))
#all_fields = [0,2,4,7,9,10,12,13,14,15]
level_table, cols = createTable(all_fields, cands)

# Get the unique values used in the initial level data.
level_fields = [l[2:] for l in level_table]
for i in range(0,max([len(l) for l in level_fields])):
        m = set([l[i] for l in level_fields])
        print(cols[2:][i], sorted(m))

Byte 1  ['$00', '$02', '$03', '$04', '$05', '$06', '$07', '$08', '$09', '$0A', '$0B', '$0C', '$0D', '$0E', '$0F', '$10', '$11', '$55']
Byte 3  ['$FC', 'ATARI_ST', 'BALLOON', 'BIRD1', 'BOLAS1', 'BOUNCY_RING', 'BUBBLE', 'BULLHEAD', 'CAMEL', 'COPTIC_CROSS', 'CUMMING_COCK1', 'EYE_OF_HORUS', 'FLAG_BAR', 'FLYING_COCK1', 'FLYING_COCK_RIGHT1', 'FLYING_COMMA1', 'FLYING_DART1', 'FLYING_DOT1', 'FLYING_FLOWCHART1', 'FLYING_SAUCER1', 'FLYING_TRIANGLE1', 'GILBY_AIRBORNE_LEFT', 'GILBY_AIRBORNE_RIGHT', 'LAND_GILBY1', 'LICKER_SHIP1', 'LITTLE_DART', 'LITTLE_OTHER_EYEBALL', 'LLAMA', 'LOZENGE', 'MAGIC_MUSHROOM', 'PSI', 'QBERT_SQUARES', 'SMALLBALL_AGAIN', 'SMALL_BALL1', 'SPINNING_RING1', 'STARSHIP', 'STRANGE_SYMBOL', 'TEARDROP_EXPLOSION1', 'WINGBALL']
Byte 4  ['$00', '$01', '$02', '$03', '$04', '$05', '$06', '$07']
Byte 6  ['$FC', 'ATARI_ST', 'BIRD1', 'BOLAS1', 'BOUNCY_RING', 'BUBBLE', 'BULLHEAD', 'CAMEL', 'COPTIC_CROSS', 'EYE_OF_HORUS', 'FLAG_BAR', 'FLYING_COCK1', 'FLYING_COCK_RIGHT1', 'FLYING_COMMA1', 'F

In [245]:
#all_fields = list(range(0,30))
all_fields = [5,11,18,19,20,21]
level_table, cols = createTable(all_fields, cands)


In [246]:
df = pd.DataFrame(level_table, columns=cols)
#df = df.set_axis(sorted(cands.keys()), axis='index')

df = df.style.set_caption("Level Data in Iridis Alpha").set_table_styles(styles)
display(df)


,Planet,Level,Byte 9,Byte 18,Byte 26,Byte 28,Byte 30,Byte 32
0,1,1,nullPtr,planet1Level1Data2ndStage,nullPtr,nullPtr,spinningRings,defaultExplosion
1,1,2,nullPtr,nullPtr,nullPtr,planet1Level2Data,spinningRings,planet1Level2Data
2,1,3,nullPtr,planet1Level3Data2ndStage,nullPtr,nullPtr,lickerShipWaveData,lickerShipWaveData
3,1,4,nullPtr,planet1Level4Data2ndStage,nullPtr,nullPtr,planet1Level4Data2ndStage,planet1Level4Data2ndStage
4,1,5,nullPtr,nullPtr,nullPtr,planet1Level5Data2ndStage,planet1Level5Data3rdStage,defaultExplosion
5,1,6,nullPtr,planet1Level6Data2ndStage,nullPtr,nullPtr,spinningRings2ndType,defaultExplosion
6,1,7,nullPtr,planet1Level7Data2ndStage,nullPtr,nullPtr,planet1Level7Data2ndStage,defaultExplosion
7,1,8,nullPtr,nullPtr,nullPtr,nullPtr,planet1Level8Data2ndStage,planet1Level8Data2ndStage
8,1,9,nullPtr,planet1Level9DataSecondStage,planet1Level9DataSecondStage,nullPtr,defaultExplosion,defaultExplosion
9,1,10,nullPtr,planet1Level10Data2ndStage,nullPtr,planet1Level10Data,planet1Level10Data2ndStage,defaultExplosion


## Wave Data Pointers in Level Data

In [251]:
#all_fields = list(range(0,30))
all_fields = [5,11,18,19,20,21]
level_table, cols = createTable(all_fields, cands)

rows = level_table
#print('\nTabulate Latex:')
print(tabulate(level_table, headers=cols, tablefmt='latex'))

#print('\nTexttable Latex:')
#print(latextable.draw_latex(table, caption="A comparison of rocket features."))

\begin{tabular}{rrllllll}
\hline
   Planet &   Level & Byte 9                   & Byte 18                      & Byte 26                      & Byte 28                   & Byte 30                    & Byte 32                    \\
\hline
        1 &       1 & nullPtr                  & planet1Level1Data2ndStage    & nullPtr                      & nullPtr                   & spinningRings              & defaultExplosion           \\
        1 &       2 & nullPtr                  & nullPtr                      & nullPtr                      & planet1Level2Data         & spinningRings              & planet1Level2Data          \\
        1 &       3 & nullPtr                  & planet1Level3Data2ndStage    & nullPtr                      & nullPtr                   & lickerShipWaveData         & lickerShipWaveData         \\
        1 &       4 & nullPtr                  & planet1Level4Data2ndStage    & nullPtr                      & nullPtr                   & planet1Level4Data2ndStage  & 

## Settings in Level Data

In [354]:
#all_fields = list(range(0,30))
sprite_detail = [0,1,2,3]
basic_movement = [4,12,13,14,15]
enemy_behaviour = [10,16,17,24,25]
pointer_data = [5,11,18,19,20,21]

header = """
\\begin{figure}[H]
  {
  \\setlength{\\tabcolsep}{3.0pt}
  \\setlength\\cmidrulewidth{\\heavyrulewidth} % Make cmidrule = 
  \\begin{adjustbox}{width=\\textwidth}
"""

footer = """
  \\end{adjustbox}

  }\\caption*{%CAPTION%.}
\\end{figure}
"""

byte_descriptions = {
"Level" : "",
"Byte 1"  : "Index into array for sprite color",                            
"Byte 10" : "Lo Ptr for an animation effect? (Doesn't seem to be used?)",                   
"Byte 11" : "Hi Ptr for an animation effect (Doesn't seem to be used?)?",                   
"Byte 12" : "some kind of rate limiting for attack wave",                                   
"Byte 13" : "Lo Ptr for a stage in wave data (never used).",                                
"Byte 14" : "Hi Ptr for a stage in wave data (never used).",                                
"Byte 15" : "Unused, see GetNewShipDataFromDataStore",                                      
"Byte 16" : "Update rate for attack wave",                                                  
"Byte 17" : "Lo Ptr to the wave data we switch to when first hit.",                         
"Byte 18" : "Hi Ptr to the wave data we switch to when first hit.",                         
"Byte 19" : "X Pos movement for attack ship.",                                              
"Byte 2"  : "Sprite value for the attack ship for the upper planet.",                       
"Byte 20" : "Y Pos movement pattern for attack ship.",                                      
"Byte 21" : "X Pos Frame Rate for Attack ship.",                                            
"Byte 22" : "Y Pos Frame Rate for Attack ship.",                                            
"Byte 23" : "Stickiness factor, does the enemy stick to the player",                        
"Byte 24" : "Does the enemy gravitate quickly toward the player when its hit?",                  
"Byte 25" : "Lo Ptr for another set of wave data.",                                         
"Byte 26" : "Hi Ptr for another set of wave data.",                                         
"Byte 27" : "Lo Ptr for another set of wave data.",                                         
"Byte 28" : "Hi Ptr for another set of wave data.",                                         
"Byte 29" : "Lo Ptr for Explosion animation.",                                              
"Byte 3"  : "Sprite value for the attack ship on the upper planet",                       
"Byte 30" : "Hi Ptr for Explosion animation.",                                              
"Byte 31" : "Lo Ptr for another set of wave data for this level.",                          
"Byte 32" : "Hi Ptr for another set of wave data for this level.",                          
"Byte 33" : "Unused.",                                                                      
"Byte 34" : "Unused, see GetNewShipDataFromDataStore.",                                     
"Byte 35" : "Does destroying this enemy increase the gilby's energy?.",                     
"Byte 36" : "Does colliding with this enemy decrease the gilby's energy?",                  
"Byte 37": "Is the ship a spinning ring, i.e. does it allow the gilby to warp?",
"Byte 38": "Number of waves in data.",                                          
"Byte 39": "Number of ships in wave.",                                          
"Byte 4" : "The animation frame rate for the attack ship.",                                 
"Byte 40": "Unused bytes.",                                                     
"Byte 5" : "Sprite for the attack ship for the lower planet.",                        
"Byte 6" : "Sprite value for the attack ship on lower planet",                 
"Byte 7" : "Whether a specific attack behaviour is used.",                                  
"Byte 8" : "Lo Ptr for an unused attack behaviour",                                         
"Byte 9" : "Lo and Hi Ptr for alternate enemy mode",                                         
}

adl = "\\addlinespace"
leg = "\\multicolumn{%len%}{@{}l@{}}{%BYTE%}\\\\"


### Sprite Data

In [313]:

level_table, cols = createTable(sprite_detail, cands)
for i in range(1,6):
    lt = [l[1:] for l in level_table if l[0] == str(i)]
    print(header)
    
    ltable = tabulate(lt, headers=cols[1:], tablefmt='latex_booktabs')
    leg_cols = [f"{c}: {byte_descriptions[c.strip()]}" for c in cols[2:]]
    legend = '\n'.join([leg.replace("%len%", str(len(leg_cols)))
                        .replace("%BYTE%", lc) for lc in leg_cols])
    ltable = ltable.replace("\\bottomrule", "\\addlinespace\n\\bottomrule\n" + legend)

    print(ltable)
    print(footer.replace("%CAPTION%", f"Planet {i} - Sprite Data"))


\begin{figure}[H]
  {
  \setlength{\tabcolsep}{3.0pt}
  \setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule = 
  \begin{adjustbox}{width=\textwidth}

\begin{tabular}{rllll}
\toprule
   Level & Byte 1    & Byte 3               & Byte 4    & Byte 6                           \\
\midrule
       1 & \$06       & FLYING\_SAUCER1       & \$03       & FLYING\_SAUCER1                   \\
       2 & \$06       & BOUNCY\_RING          & \$01       & BOUNCY\_RING                      \\
       3 & \$05       & FLYING\_DOT1          & \$04       & FLYING\_DOT1                      \\
       4 & \$11       & FLYING\_TRIANGLE1     & \$03       & FLYING\_TRIANGLE1                 \\
       5 & \$11       & BALLOON              & \$00       & BIRD1                            \\
       6 & \$0A       & BIRD1                & \$03       & BIRD1                            \\
       7 & \$09       & FLAG\_BAR             & \$00       & FLAG\_BAR                         \\
       8 & \$11       & TE

### Basic Movement

In [334]:
basic_movement = [4,12,13,14,15]
level_table, cols = createTable(basic_movement, cands)
for i in range(1,6):
    lt = [l[1:] for l in level_table if l[0] == str(i)]
    print(header)
    
    ltable = tabulate(lt, headers=cols[1:], tablefmt='latex_booktabs')
    leg_cols = [f"{c}: {byte_descriptions[c.strip()]}" for c in cols[2:]]
    legend = '\n'.join([leg.replace("%len%", str(len(leg_cols)+1))
                        .replace("%BYTE%", lc) for lc in leg_cols])
    ltable = ltable.replace("\\bottomrule", "\\addlinespace\n\\bottomrule\n" + legend)

    print(ltable)
    print(footer.replace("%CAPTION%", f"Planet {i} - Movement Data"))


\begin{figure}[H]
  {
  \setlength{\tabcolsep}{3.0pt}
  \setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule = 
  \begin{adjustbox}{width=\textwidth}

\begin{tabular}{rlllll}
\toprule
   Level & Byte 7    & Byte 19   & Byte 20   & Byte 21   & Byte 22   \\
\midrule
       1 & \$00       & \$06       & \$01       & \$01       & \$01       \\
       2 & \$00       & \$00       & \$24       & \$02       & \$01       \\
       3 & \$00       & \$FA       & \$01       & \$01       & \$02       \\
       4 & \$00       & \$07       & \$00       & \$01       & \$02       \\
       5 & \$00       & \$FC       & \$23       & \$02       & \$02       \\
       6 & \$00       & \$00       & \$00       & \$01       & \$01       \\
       7 & \$00       & \$07       & \$00       & \$01       & \$02       \\
       8 & \$00       & \$05       & \$00       & \$01       & \$02       \\
       9 & \$00       & \$FC       & \$23       & \$01       & \$03       \\
      10 & \$00       & \$00       &

### Enemy Behaviour

In [331]:
enemy_behaviour = [10,16,17,24,25,26,27,28]

level_table, cols = createTable(enemy_behaviour, cands)
for i in range(1,6):
    lt = [l[1:] for l in level_table if l[0] == str(i)]
    print(header)
    
    ltable = tabulate(lt, headers=cols[1:], tablefmt='latex_booktabs')
    leg_cols = [f"{c}: {byte_descriptions[c.strip()]}" for c in cols[2:]]
    legend = '\n'.join([leg.replace("%len%", str(len(leg_cols)))
                        .replace("%BYTE%", lc) for lc in leg_cols])
    ltable = ltable.replace("\\bottomrule", "\\addlinespace\n\\bottomrule\n" + legend)

    print(ltable)
    print(footer.replace("%CAPTION%", f"Planet {i} - Enemy Behaviour Data"))


\begin{figure}[H]
  {
  \setlength{\tabcolsep}{3.0pt}
  \setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule = 
  \begin{adjustbox}{width=\textwidth}

\begin{tabular}{rllllllll}
\toprule
   Level & Byte 16   & Byte 23   & Byte 24   & Byte 35   & Byte 36   & Byte 37   & Byte 38   & Byte 39   \\
\midrule
       1 & \$40       & \$00       & \$00       & \$02       & \$02       & \$00       & \$04       & \$18       \\
       2 & \$00       & \$01       & \$23       & \$01       & \$01       & \$00       & \$04       & \$20       \\
       3 & \$30       & \$00       & \$00       & \$02       & \$01       & \$00       & \$04       & \$20       \\
       4 & \$60       & \$00       & \$01       & \$04       & \$02       & \$00       & \$04       & \$20       \\
       5 & \$00       & \$00       & \$23       & \$00       & \$05       & \$00       & \$04       & \$20       \\
       6 & \$03       & \$01       & \$01       & \$01       & \$04       & \$00       & \$04       & \$10    

### Pointer Data

In [322]:
pointer_data = [5,11,18,19,20,21]

level_table, cols = createTable(pointer_data, cands)
for i in range(1,6):
    lt = [l[1:] for l in level_table if l[0] == str(i)]
    print(header)
    
    ltable = tabulate(lt, headers=cols[1:], tablefmt='latex_booktabs')
    leg_cols = [f"{c}: {byte_descriptions[c.strip()]}" for c in cols[2:]]
    legend = '\n'.join([leg.replace("%len%", str(len(leg_cols)))
                        .replace("%BYTE%", lc) for lc in leg_cols])
    ltable = ltable.replace("\\bottomrule", "\\addlinespace\n\\bottomrule\n" + legend)

    print(ltable)
    print(footer.replace("%CAPTION%", f"Planet {i} - Pointer Data"))


\begin{figure}[H]
  {
  \setlength{\tabcolsep}{3.0pt}
  \setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule = 
  \begin{adjustbox}{width=\textwidth}

\begin{tabular}{rllllll}
\toprule
   Level & Byte 9            & Byte 18                      & Byte 26                      & Byte 28                   & Byte 30                    & Byte 32                    \\
\midrule
       1 & nullPtr           & planet1Level1Data2ndStage    & nullPtr                      & nullPtr                   & spinningRings              & defaultExplosion           \\
       2 & nullPtr           & nullPtr                      & nullPtr                      & planet1Level2Data         & spinningRings              & planet1Level2Data          \\
       3 & nullPtr           & planet1Level3Data2ndStage    & nullPtr                      & nullPtr                   & lickerShipWaveData         & lickerShipWaveData         \\
       4 & nullPtr           & planet1Level4Data2ndStage    & nullPtr           

### Bytes 7,8,9

In [355]:
pointer_data = [4,5]

level_table, cols = createTable(pointer_data, cands)
lt = [l for l in level_table if l[2] != "$00"]
print(header)

ltable = tabulate(lt, headers=cols[1:], tablefmt='latex_booktabs')
leg_cols = [f"{c}: {byte_descriptions[c.strip()]}" for c in cols[2:]]
legend = '\n'.join([leg.replace("%len%", str(len(leg_cols)))
                    .replace("%BYTE%", lc) for lc in leg_cols])
ltable = ltable.replace("\\bottomrule", "\\addlinespace\n\\bottomrule\n" + legend)

print(ltable)
print(footer.replace("%CAPTION%", f"Planet {i} - Pointer Data"))


\begin{figure}[H]
  {
  \setlength{\tabcolsep}{3.0pt}
  \setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule = 
  \begin{adjustbox}{width=\textwidth}

\begin{tabular}{rrll}
\toprule
    &   Level & Byte 7    & Byte 9                   \\
\midrule
  1 &      11 & \$03       & smallDotWaveData         \\
  1 &      14 & \$03       & planet1Level8Data        \\
  2 &      19 & \$0C       & landGilbyAsEnemy         \\
  3 &       4 & \$04       & gilbyLookingLeft         \\
  3 &       6 & \$04       & planet3Level6Additional  \\
  4 &      19 & \$01       & planet4Level19Additional \\
  5 &       3 & \$01       & planet5Level3Additional  \\
  5 &       5 & \$05       & planet5Level5Additional  \\
  5 &      14 & \$06       & llamaWaveData            \\
\addlinespace
\bottomrule
\multicolumn{2}{@{}l@{}}{Byte 7 : Whether a specific attack behaviour is used.}\\
\multicolumn{2}{@{}l@{}}{Byte 9 : Lo and Hi Ptr for alternate enemy mode}\\
\end{tabular}

  \end{adjustbox}

  }\caption*{Pla